Here we will be doing data_engineering for the regression task

In [87]:
#importing library and reading documents
import pandas as pd

df = pd.read_csv("C:/Users/Ralph Arren/Desktop/Uni/Uni Year 3/ML_Project/cleaned_nyc_capital_projects.csv")
df.head()

,Fiscal_Year,Borough,Award,Title,Description,Budget_Line,Council_District_num
0,FY19,X,250000,VOLUNTEERS OF AMERICA,CLARKE PLACE SENIOR RESIDENCE PROJECT,HD D024,16.0
1,FY19,K,3640000,CYPRESS HILLS CHILD CARE CENTER,CONSTRUCTION OF A CHILD CARE CENTER,CS DN956,37.0
2,FY19,M,425000,[SCA] [02M077/02M198] LOWER LAB/ISADOR E. IDA ...,HVAC SYSTEM FOR CAFETERIA,E D001,5.0
3,FY19,M,100000,[SCA] [02M077/02M198] LOWER LAB/ISADOR E. IDA ...,PLAYGROUND RENOVATION,E D001,5.0
4,FY19,M,35000,[SCA] [02M114] EAST SIDE MIDDLE SCHOOL,TECHNOLOGY UPGRADES,E D001,5.0


_________________________________________________________________________________________________________________________________
Data Engineering: Regression - Predict Annual Capital Spending Per Borough
- Borough
- Fiscal_Year to be converted to numeric
- Project Count per Borough per year
- Total Award
- Avg Award
- Median, IQR (Q3-Q1)


In [88]:
#Converting Fiscal Years to numeric
df['Fiscal_Year'] = ('20' + df['Fiscal_Year'].str.replace('FY', '', regex=False)).astype(int)

#check
df.head()

,Fiscal_Year,Borough,Award,Title,Description,Budget_Line,Council_District_num
0,2019,X,250000,VOLUNTEERS OF AMERICA,CLARKE PLACE SENIOR RESIDENCE PROJECT,HD D024,16.0
1,2019,K,3640000,CYPRESS HILLS CHILD CARE CENTER,CONSTRUCTION OF A CHILD CARE CENTER,CS DN956,37.0
2,2019,M,425000,[SCA] [02M077/02M198] LOWER LAB/ISADOR E. IDA ...,HVAC SYSTEM FOR CAFETERIA,E D001,5.0
3,2019,M,100000,[SCA] [02M077/02M198] LOWER LAB/ISADOR E. IDA ...,PLAYGROUND RENOVATION,E D001,5.0
4,2019,M,35000,[SCA] [02M114] EAST SIDE MIDDLE SCHOOL,TECHNOLOGY UPGRADES,E D001,5.0


In [89]:
df[['Budget_Line', 'Description']]

,Budget_Line,Description
0,HD D024,CLARKE PLACE SENIOR RESIDENCE PROJECT
1,CS DN956,CONSTRUCTION OF A CHILD CARE CENTER
2,E D001,HVAC SYSTEM FOR CAFETERIA
3,E D001,PLAYGROUND RENOVATION
4,E D001,TECHNOLOGY UPGRADES
...,...,...
11486,HL DN404,ELECTROPHYSIOLOGY LAB 2
11487,HR DN918,TECHNICAL ADJUSTMENT TO RESTORE FUNDS TO PROJE...
11488,ED DN690,TECHNICAL ADJUSTMENT TO RESTORE FUNDS TO 850 E...
11489,HH DN336,MOBILE HEALTHCARE VEHICLE


In [90]:
budget_types = df['Budget_Line'].str.split(' ').str[0]
print(sorted(budget_types.unique()))

['AG', 'CO', 'CS', 'E', 'ED', 'EP', 'F', 'HA', 'HD', 'HH', 'HL', 'HN', 'HO', 'HR', 'HW', 'L', 'LB', 'LN', 'LQ', 'P', 'PO', 'PU', 'PV', 'PW', 'S', 'SE', 'T', 'TF', 'WP']


_________________________________________________________________________________________________________________________________
Department Budget Line ID abbreviations
(source = https://databook.nyc/capital-archive/budget-lines)
- AG = Dept of Aging
- BR = Dept of Transpo; Bridges
- C = Dept of Correction
- CO = Courts
- CS = Children's Services
- DP = Energy Efficiency and Citywide Equipment
- E = Education
- ED = Small Business Service
- EP = Department of Environmental Protection; Equipment
- F = Fire Department
- FA = Department of Transportation; Ferries
- HA = Housing Authority
- HB = Department of Transportation; Bridges
- HD = Housing Preservation & Development
- HH = Homeless Services
- HL = Department of Health and Mental Hygiene
- HN = City University
- HO = Health and Hospitals Corporations
- HR = Human Resources Administration
- HW = Department of Transportation; Highways
- L = New York Research Libraries
- LB = Brooklyn Public Library
- LN = New York Public Library
- LQ = Queens Public Library
- MT = Transit Authority; MTA Bus Company
- P = Department of Parks & Recreation
- PO = Police Department
- PU = Energy Efficiency and Citywide Equipment
- PV = Department of Cultural Affairs
- PW = Department of Citywide Administrative Services; Public Buildings
- RE = Department of Citywide Administrative Services; Real Property
- S = Department of Sanitation
- SE = Department of Environmental Protection; Sewers
- T = Transit Authority; Transit
- TD = Department of Transportation; Equipment
- TF = Department of Transportation; Traffic
- W = Department of Environmental Protection; Water Supply
- WM = Department of Environmental Protection; Water Mains, Sources and Treatment
- WP = Department of Environmental Protection; Water Pollution Control

We will be categorizing these in order to be used for one-hot encoding. This is because knowing what an award goes to (for example health, trasnport, city facilities) is important for predicting how awards and for predicting how much a project will get.

In [91]:
#mapping
sector_map = {
    # Transportation
    'BR': 'Transportation', 'HB': 'Transportation', 'HW': 'Transportation',
    'FA': 'Transportation', 'TD': 'Transportation', 'TF': 'Transportation',
    'MT': 'Transportation', 'T': 'Transportation',
    
    # Environment & Utility
    'EP': 'Environment', 'SE': 'Environment', 'W': 'Environment',
    'WM': 'Environment', 'WP': 'Environment', 'S': 'Environment',
    'DP': 'Environment', 'PU': 'Environment',
    
    # Education
    'E': 'Education', 'HN': 'Education',
    
    # Public Safety & Justice
    'C': 'Public Safety', 'CO': 'Public Safety', 
    'F': 'Public Safety', 'PO': 'Public Safety',
    
    # Health & Human Services
    'AG': 'Health & Human Svcs', 'CS': 'Health & Human Svcs', 
    'HH': 'Health & Human Svcs', 'HL': 'Health & Human Svcs', 
    'HO': 'Health & Human Svcs', 'HR': 'Health & Human Svcs',
    
    # Housing & Econ Development
    'HA': 'Housing & Dev', 'HD': 'Housing & Dev', 
    'ED': 'Housing & Dev', 'RE': 'Housing & Dev',
    
    # Community (Parks, Libraries, Culture)
    'L': 'Community & Culture', 'LB': 'Community & Culture', 
    'LN': 'Community & Culture', 'LQ': 'Community & Culture',
    'P': 'Community & Culture', 'PV': 'Community & Culture',
    
    # General Admin
    'PW': 'General Admin'
}

In [92]:
#using with dataframe

df['Sector'] = df['Budget_Line'].str.split(' ').str[0].map(sector_map)

In [93]:
#checking
df

,Fiscal_Year,Borough,Award,Title,Description,Budget_Line,Council_District_num,Sector
0,2019,X,250000,VOLUNTEERS OF AMERICA,CLARKE PLACE SENIOR RESIDENCE PROJECT,HD D024,16.0,Housing & Dev
1,2019,K,3640000,CYPRESS HILLS CHILD CARE CENTER,CONSTRUCTION OF A CHILD CARE CENTER,CS DN956,37.0,Health & Human Svcs
2,2019,M,425000,[SCA] [02M077/02M198] LOWER LAB/ISADOR E. IDA ...,HVAC SYSTEM FOR CAFETERIA,E D001,5.0,Education
3,2019,M,100000,[SCA] [02M077/02M198] LOWER LAB/ISADOR E. IDA ...,PLAYGROUND RENOVATION,E D001,5.0,Education
4,2019,M,35000,[SCA] [02M114] EAST SIDE MIDDLE SCHOOL,TECHNOLOGY UPGRADES,E D001,5.0,Education
...,...,...,...,...,...,...,...,...
11486,2026,R,933000,STATEN ISLAND UNIVERSITY HOSPITAL,ELECTROPHYSIOLOGY LAB 2,HL DN404,99.0,Health & Human Svcs
11487,2026,Q,318000,"KOREAN-AMERICAN FAMILY SERVICE CENTER, INC. (8...",TECHNICAL ADJUSTMENT TO RESTORE FUNDS TO PROJE...,HR DN918,99.0,Health & Human Svcs
11488,2026,M,189000,"CEC STUYVESANT COVE, INC. DBA SOLAR ONE (850 E...",TECHNICAL ADJUSTMENT TO RESTORE FUNDS TO 850 E...,ED DN690,99.0,Housing & Dev
11489,2026,M,111000,PROJECT RENEWAL VEHICLE,MOBILE HEALTHCARE VEHICLE,HH DN336,99.0,Health & Human Svcs


In [94]:
df_encoded = pd.get_dummies(df, columns=['Sector'], drop_first=False)
sector_cols = [col for col in df_encoded.columns if col.startswith('Sector_')]
df_encoded[sector_cols] = df_encoded[sector_cols].astype(int)

df_encoded.head()

,Fiscal_Year,Borough,Award,Title,Description,Budget_Line,Council_District_num,Sector_Community & Culture,Sector_Education,Sector_Environment,Sector_General Admin,Sector_Health & Human Svcs,Sector_Housing & Dev,Sector_Public Safety,Sector_Transportation
0,2019,X,250000,VOLUNTEERS OF AMERICA,CLARKE PLACE SENIOR RESIDENCE PROJECT,HD D024,16.0,0,0,0,0,0,1,0,0
1,2019,K,3640000,CYPRESS HILLS CHILD CARE CENTER,CONSTRUCTION OF A CHILD CARE CENTER,CS DN956,37.0,0,0,0,0,1,0,0,0
2,2019,M,425000,[SCA] [02M077/02M198] LOWER LAB/ISADOR E. IDA ...,HVAC SYSTEM FOR CAFETERIA,E D001,5.0,0,1,0,0,0,0,0,0
3,2019,M,100000,[SCA] [02M077/02M198] LOWER LAB/ISADOR E. IDA ...,PLAYGROUND RENOVATION,E D001,5.0,0,1,0,0,0,0,0,0
4,2019,M,35000,[SCA] [02M114] EAST SIDE MIDDLE SCHOOL,TECHNOLOGY UPGRADES,E D001,5.0,0,1,0,0,0,0,0,0


In [95]:
df_encoded.drop('Budget_Line', axis=1)
df_encoded

,Fiscal_Year,Borough,Award,Title,Description,Budget_Line,Council_District_num,Sector_Community & Culture,Sector_Education,Sector_Environment,Sector_General Admin,Sector_Health & Human Svcs,Sector_Housing & Dev,Sector_Public Safety,Sector_Transportation
0,2019,X,250000,VOLUNTEERS OF AMERICA,CLARKE PLACE SENIOR RESIDENCE PROJECT,HD D024,16.0,0,0,0,0,0,1,0,0
1,2019,K,3640000,CYPRESS HILLS CHILD CARE CENTER,CONSTRUCTION OF A CHILD CARE CENTER,CS DN956,37.0,0,0,0,0,1,0,0,0
2,2019,M,425000,[SCA] [02M077/02M198] LOWER LAB/ISADOR E. IDA ...,HVAC SYSTEM FOR CAFETERIA,E D001,5.0,0,1,0,0,0,0,0,0
3,2019,M,100000,[SCA] [02M077/02M198] LOWER LAB/ISADOR E. IDA ...,PLAYGROUND RENOVATION,E D001,5.0,0,1,0,0,0,0,0,0
4,2019,M,35000,[SCA] [02M114] EAST SIDE MIDDLE SCHOOL,TECHNOLOGY UPGRADES,E D001,5.0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11486,2026,R,933000,STATEN ISLAND UNIVERSITY HOSPITAL,ELECTROPHYSIOLOGY LAB 2,HL DN404,99.0,0,0,0,0,1,0,0,0
11487,2026,Q,318000,"KOREAN-AMERICAN FAMILY SERVICE CENTER, INC. (8...",TECHNICAL ADJUSTMENT TO RESTORE FUNDS TO PROJE...,HR DN918,99.0,0,0,0,0,1,0,0,0
11488,2026,M,189000,"CEC STUYVESANT COVE, INC. DBA SOLAR ONE (850 E...",TECHNICAL ADJUSTMENT TO RESTORE FUNDS TO 850 E...,ED DN690,99.0,0,0,0,0,0,1,0,0
11489,2026,M,111000,PROJECT RENEWAL VEHICLE,MOBILE HEALTHCARE VEHICLE,HH DN336,99.0,0,0,0,0,1,0,0,0


_________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
Dealing with Description and Title :
- Budget_Line_IDs tells us WHO is spending
- Description and Title might tell us WHAT we are spending on

In [96]:
description_labels = df['Description']
print(description_labels.unique())
print(len(description_labels))

['CLARKE PLACE SENIOR RESIDENCE PROJECT'
 'CONSTRUCTION OF A CHILD CARE CENTER' 'HVAC SYSTEM FOR CAFETERIA' ...
 'TECHNICAL ADJUSTMENT TO RESTORE FUNDS TO 850 EDNSOLAR1'
 'MOBILE HEALTHCARE VEHICLE' 'EXTERIOR AND INTERIOR UPGRADES']
11491


In [97]:
from sklearn.preprocessing import MultiLabelBinarizer
# Combine Title and Description for easier text searching
combined_text = (df_encoded['Title'].fillna('') + ' ' + df_encoded['Description'].fillna('')).str.lower()

# 3. Define the mapping dictionary
categories = {
    'Categ_Technology': ['tech', 'computer', 'smartboard', 'laptop', 'software', 'it upgrade', 'broadband', 'a+ stem', 'stem lab'],
    'Categ_Education': ['school', 'college', 'university', 'student', 'classroom', 'academy', 'library', 'education', 'child care', 'sca'],
    'Categ_Healthcare': ['hospital', 'medical', 'health', 'clinic', 'ambulance', 'patient', 'nursing', 'c-arm'],
    'Categ_Parks & Recreation': ['park', 'playground', 'garden', 'recreation', 'athletic', 'field', 'court', 'pool'],
    'Categ_Arts & Culture': ['museum', 'theater', 'arts', 'culture', 'cultural', 'music', 'dance', 'performance'],
    'Categ_Infrastructure/Renovation': ['renovation', 'upgrade', 'hvac', 'boiler', 'roof', 'bathroom', 'auditorium', 'elevator', 'construction', 'repair', 'facility'],
    'Categ_Community/Housing': ['housing', 'senior', 'community', 'hunger', 'shelter', 'food', 'affordable', 'neighborhood'],
    'Categ_Safety/Security': ['police', 'nypd', 'fdny', 'fire', 'camera', 'security', 'safety']
}

# Function to assign categories based on keywords
def assign_categories(text):
    cats = []
    for cat, keywords in categories.items():
        if any(kw in text for kw in keywords):
            cats.append(cat)
    if not cats:
        cats.append('Other')
    return cats

# Apply the function
df_encoded['Categories'] = combined_text.apply(assign_categories)

# 5. One-Hot Encode the multi-label categories
mlb = MultiLabelBinarizer()
one_hot_encoded_df = pd.DataFrame(mlb.fit_transform(df_encoded['Categories']), columns=mlb.classes_, index=df_encoded.index)

# Join the one-hot encoded columns back to the original DataFrame
final_df = pd.concat([df_encoded, one_hot_encoded_df], axis=1)

# Drop the temporary 'Text' and 'Categories' columns if desired

# Inspect the results
print(one_hot_encoded_df.sum().sort_values(ascending=False))

Categ_Infrastructure/Renovation    6690
Categ_Technology                   4097
Categ_Education                    3659
Categ_Parks & Recreation           1557
Other                              1158
Categ_Arts & Culture                877
Categ_Community/Housing             857
Categ_Safety/Security               582
Categ_Healthcare                    501
dtype: int64


In [98]:
final_df.columns.tolist()

['Fiscal_Year',
 'Borough',
 'Award',
 'Title',
 'Description',
 'Budget_Line',
 'Council_District_num',
 'Sector_Community & Culture',
 'Sector_Education',
 'Sector_Environment',
 'Sector_General Admin',
 'Sector_Health & Human Svcs',
 'Sector_Housing & Dev',
 'Sector_Public Safety',
 'Sector_Transportation',
 'Categories',
 'Categ_Arts & Culture',
 'Categ_Community/Housing',
 'Categ_Education',
 'Categ_Healthcare',
 'Categ_Infrastructure/Renovation',
 'Categ_Parks & Recreation',
 'Categ_Safety/Security',
 'Categ_Technology',
 'Other']

In [101]:
final_df.columns.tolist()

['Fiscal_Year',
 'Borough',
 'Award',
 'Title',
 'Description',
 'Budget_Line',
 'Council_District_num',
 'Sector_Community & Culture',
 'Sector_Education',
 'Sector_Environment',
 'Sector_General Admin',
 'Sector_Health & Human Svcs',
 'Sector_Housing & Dev',
 'Sector_Public Safety',
 'Sector_Transportation',
 'Categories',
 'Categ_Arts & Culture',
 'Categ_Community/Housing',
 'Categ_Education',
 'Categ_Healthcare',
 'Categ_Infrastructure/Renovation',
 'Categ_Parks & Recreation',
 'Categ_Safety/Security',
 'Categ_Technology',
 'Other']

In [103]:
final_df = final_df.drop(['Categories', 'Title', 'Description', 'Budget_Line'], axis=1)
final_df

,Fiscal_Year,Borough,Award,Council_District_num,Sector_Community & Culture,Sector_Education,Sector_Environment,Sector_General Admin,Sector_Health & Human Svcs,Sector_Housing & Dev,...,Sector_Transportation,Categ_Arts & Culture,Categ_Community/Housing,Categ_Education,Categ_Healthcare,Categ_Infrastructure/Renovation,Categ_Parks & Recreation,Categ_Safety/Security,Categ_Technology,Other
0,2019,X,250000,16.0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
1,2019,K,3640000,37.0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,0
2,2019,M,425000,5.0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,2019,M,100000,5.0,0,1,0,0,0,0,...,0,0,0,1,0,1,1,0,0,0
4,2019,M,35000,5.0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11486,2026,R,933000,99.0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,0,0
11487,2026,Q,318000,99.0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
11488,2026,M,189000,99.0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
11489,2026,M,111000,99.0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [104]:
final_df.to_csv('engineeed_1_nyc.csv', index=False)